In [ ]:
import cv2 as cv
import numpy as np
import math
import sys
import os
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/ForAllOneK'
kmeans_path = os.path.join(path, 'Original', 'kmeans')
sketch_outline_path = os.path.join(path, 'Sketch-Rendered','sketch-rendered')

In [ ]:
sketch_list = ['width-1', 'width-3', 'width-5']
"""for f in sketch_list:
  if not os.path.isdir(os.path.join(sketch_outline_path, f, 'processed')):
    os.mkdir(os.path.join(sketch_outline_path, f, 'processed'))"""

"for f in sketch_list:\n  if not os.path.isdir(os.path.join(sketch_outline_path, f, 'processed')):\n    os.mkdir(os.path.join(sketch_outline_path, f, 'processed'))"

In [ ]:
def MASKING(image_kmean, image_outline):
  l1,b1,_ = image_kmean.shape 
  l2,b2,_ = image_outline.shape
  l = min(l1,l2)
  b = min(b1,b2)
  image_kmean = image_kmean[:l, :b, :]
  image_outline = image_outline[:l, :b, :]
  gray_outline = cv.cvtColor(image_outline, cv.COLOR_BGR2GRAY)
  _, mask = cv.threshold(gray_outline, 200, 255, cv.THRESH_BINARY)
  inv_mask = cv.bitwise_not(mask)
  white_bg = np.zeros(image_kmean.shape, np.uint8)
  white_bg.fill(255)
  colored_outline_on_black = cv.bitwise_or(image_kmean, image_kmean, mask=inv_mask)
  img1=white_bg.copy()
  img2=colored_outline_on_black.copy()
  img_2_shape = img2.shape
  roi = img1[0:img_2_shape[0],0:img_2_shape[1]]
  img2gray = cv.cvtColor(img2,cv.COLOR_BGR2GRAY)
  ret, mask = cv.threshold(img2gray, 10, 255, cv.THRESH_BINARY)
  mask_inv = cv.bitwise_not(mask)

  img1_bg = cv.bitwise_and(roi,roi,mask = mask_inv)
  img2_fg = cv.bitwise_and(img2,img2,mask = mask)
  dst = cv.add(img1_bg,img2_fg)
  img1[0:img_2_shape[0], 0:img_2_shape[1]] = dst
  colored_outline_on_white = img1.copy()
  return colored_outline_on_black, colored_outline_on_white

In [ ]:
for img in tqdm(os.listdir(kmeans_path)):
  kmeaned_img = cv.imread(os.path.join(kmeans_path, img))
  for wid_dir in sketch_list:
    for i in range(1,6):
      sketch_file_name = str(img.split('.')[0])+'_0'+str(i)+'.png'
      sketch_img = cv.imread(os.path.join(sketch_outline_path, wid_dir, sketch_file_name))
      try:
        b, w = MASKING(kmeaned_img, sketch_img)
      except:
        print(os.path.join(path, wid_dir, sketch_file_name))
        print(os.path.join(k_path, img))
        exit()
      save_file_path = os.path.join(sketch_outline_path, wid_dir, 'processed')
      if not os.path.isdir(os.path.join(save_file_path, "black")):
        os.mkdir(os.path.join(save_file_path, "black"))
      if not os.path.isdir(os.path.join(save_file_path, "white")):
        os.mkdir(os.path.join(save_file_path, "white"))
      w_save_file_path = os.path.join(save_file_path, "white")
      b_save_file_path = os.path.join(save_file_path, "black")
      w_processed_file_path = os.path.join(w_save_file_path, sketch_file_name.split('.')[0])
      b_processed_file_path = os.path.join(b_save_file_path, sketch_file_name.split('.')[0])
      cv.imwrite(b_processed_file_path+'.png', b)
      cv.imwrite(w_processed_file_path+'.png', w)


100%|██████████| 1000/1000 [1:02:50<00:00,  3.77s/it]
